In [14]:
# %load_ext autoreload
# %autoreload 2

import sys
import os
import pandas as pd
from datetime import datetime, timezone, timedelta

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import src.config as config
from src.inference import get_feature_store



fs = get_feature_store()

# ✅ Step 2: Read data from the feature group
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION
)
df = fg.read()

2025-05-11 02:41:42,856 INFO: Initializing external client
2025-05-11 02:41:42,857 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-11 02:41:43,991 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215676
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.08s) 


In [15]:
# Use fixed datetime since April data is not available
fixed_hour = df["pickup_hour"].dropna().max()
print(f"📅 Using most recent available hour for predictions: {fixed_hour}")


📅 Using most recent available hour for predictions: 2023-12-31 23:00:00+00:00


In [17]:
import hopsworks

# 🔐 Authenticate with Hopsworks
project = hopsworks.login()

# 🌟 Get the feature store
fs = project.get_feature_store()


fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION
)


# 📦 Read and filter data
df = fg.read()
predictions = df[df["pickup_hour"] == fixed_hour]


2025-05-11 02:43:58,936 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 02:43:58,941 INFO: Initializing external client
2025-05-11 02:43:58,942 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-11 02:43:59,989 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215676
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.85s) 


In [21]:
top_locations = predictions.sort_values("rides", ascending=False)
top_locations[["pickup_location_id", "pickup_hour", "rides"]].head(10)


pickup_location_id               pickup_hour  rides
11699            6948.10 2023-12-31 23:00:00+00:00     13
6894             6140.05 2023-12-31 23:00:00+00:00      2
10546            5329.03 2023-12-31 23:00:00+00:00      0

In [20]:
print(predictions.columns.tolist())


['pickup_hour', 'pickup_location_id', 'rides']
